魔搭社区的模型库：https://www.modelscope.cn/models/iic/nlp_gte_sentence-embedding_chinese-base

### GTE中文通用文本表示模型

文本表示是自然语言处理(NLP)领域的核心问题, 其在很多NLP、信息检索的下游任务中发挥着非常重要的作用。近几年, 随着深度学习的发展，尤其是预训练语言模型的出现极大的推动了文本表示技术的效果, 基于预训练语言模型的文本表示模型在学术研究数据、工业实际应用中都明显优于传统的基于统计模型或者浅层神经网络的文本表示模型。这里, 我们主要关注基于预训练语言模型的文本表示。

文本表示示例, 输入一个句子, 输入一个固定维度的连续向量:

输入: 吃完海鲜可以喝牛奶吗?
输出: [0.27162,-0.66159,0.33031,0.24121,0.46122,...]
文本的向量表示通常可以用于文本聚类、文本相似度计算、文本向量召回等下游任务中。

### 下载模型

In [ ]:
# 下载离线模型当当前目录，后续运行时会自动从本地加载模型
from modelscope import snapshot_download
model_dir = snapshot_download('iic/nlp_gte_sentence-embedding_chinese-base',cache_dir='./model')

### Transformer生成文本嵌入示例

In [ ]:

from sentence_transformers import SentenceTransformer
import numpy as np

# 1. 加载模型（首次运行自动下载，约4GB，建议指定缓存目录）
model = SentenceTransformer("./model/iic/nlp_gte_sentence-embedding_chinese-base")

# 2. 定义文本数据（实战中可替换为课程文档/论文摘要）
documents = [
    {
        "text": "机器学习是人工智能的核心技术，通过算法使计算机从数据中自动学习规律",
        "metadata": {"id": "doc_001", "category": "课程笔记", "source": "AI导论"}
    },
    {
        "text": "BERT模型采用双向Transformer架构，能捕捉文本上下文依赖关系",
        "metadata": {"id": "doc_002", "category": "论文摘要", "source": "NAACL2019"}
    },
    {
        "text": "FAISS支持多种索引类型，IndexIVFFlat适用于大规模向量的近似搜索",
        "metadata": {"id": "doc_003", "category": "工具手册", "source": "FAISS官方文档"}
    }
]

# 3. 生成文本嵌入
prefix = "为这个句子生成表示以用于检索相关句子："  # 任务引导前缀
texts = [prefix + doc["text"] for doc in documents]
embeddings = model.encode(
    texts,
    normalize_embeddings=True  # 向量归一化
).astype(np.float32)  # FAISS要求输入为float32类型

print(f"生成向量维度：{embeddings.shape[1]}")  # 输出：生成向量维度：768
print(f"向量数量：{embeddings.shape[0]}")  # 输出：向量数量：3

No sentence-transformers model found with name ./model/iic/nlp_gte_sentence-embedding_chinese-base. Creating a new one with mean pooling.
Loading weights: 100%|██████████| 199/199 [00:01<00:00, 119.87it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: ./model/iic/nlp_gte_sentence-embedding_chinese-base
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


生成向量维度：768
向量数量：3


### Faiss构建与保存检索示例

In [ ]:
import faiss
import json
import numpy as np
from pathlib import Path
from sentence_transformers import SentenceTransformer

# ===================== 1. 配置参数与路径 =====================
# 模型路径（本地路径，若不存在会自动从HuggingFace下载）
model_path = "./model/iic/nlp_gte_sentence-embedding_chinese-base"
# FAISS索引与元数据保存路径
data_dir = Path("./text_search_db")
data_dir.mkdir(parents=True, exist_ok=True)
index_path = data_dir / "faiss_index.index"
metadata_path = data_dir / "metadata.json"

# 检索参数
top_k = 3  # 改为返回Top-3相似结果（数据更多，便于学生观察排名）

# ===================== 2. 加载模型 =====================
print("正在加载模型...")
model = SentenceTransformer(model_path)
print("模型加载完成！")

# ===================== 3. 直接定义FAISS示例数据（无需TXT文件） =====================
# 扩充的FAISS知识点数据（大模型随机生成）
sample_data = [
    "登金陵凤凰台：凤凰台上凤凰游，凤去台空江自流。吴宫花草埋幽径，晋代衣冠成古丘。三山半落青天外，二水中分白鹭洲。总为浮云能蔽日，长安不见使人愁。",
    "赠郭将军：将军少年出武威，入掌银台护紫微。平明拂剑朝天去，薄暮垂鞭醉酒归。爱子临风吹玉笛，美人向月舞罗衣。畴昔雄豪如梦里，相逢且欲醉春晖。",
    "题雍丘崔明府丹灶：美人为尔别开池，莲花倾城出自姿。素影空中飘素月，清光水上照清辉。神仙多被宏慈误，宰执常遭白简非。借问欲栖珠树鹤，何年却向帝城飞？",
    "鹦鹉洲：鹦鹉来过吴江水，江上洲传鹦鹉名。鹦鹉西飞陇山去，芳洲之树何青青。烟开兰叶香风暖，岸夹桃花锦浪生。迁客此时徒极目，长洲孤月向谁明。",
    "别中都明府兄：吾兄诗酒继陶君，试宰中都天下闻。东楼喜奉连枝会，南陌愁为落叶分。城隅渌水明秋日，海上青山隔暮云。取醉不辞留夜月，雁行中断惜离群。",
    "将进酒：君不见黄河之水天上来，奔流到海不复回。君不见高堂明镜悲白发，朝如青丝暮成雪。人生得意须尽欢，莫使金樽空对月。天生我材必有用，千金散尽还复来。",
    "宣州谢朓楼饯别校书叔云：弃我去者昨日之日不可留，乱我心者今日之日多烦忧。长风万里送秋雁，对此可以酣高楼。蓬莱文章建安骨，中间小谢又清发。俱怀逸兴壮思飞，欲上青天览明月。",
    "梦游天姥吟留别（节选）：海客谈瀛洲，烟涛微茫信难求；越人语天姥，云霞明灭或可睹。天姥连天向天横，势拔五岳掩赤城。天台四万八千丈，对此欲倒东南倾。",
    "早发白帝城：朝辞白帝彩云间，千里江陵一日还。两岸猿声啼不住，轻舟已过万重山。",
    "黄鹤楼送孟浩然之广陵：故人西辞黄鹤楼，烟花三月下扬州。孤帆远影碧空尽，唯见长江天际流。"
]

# 解析示例数据为文档元数据列表（保持原有结构：source + text）
documents = []
for item in sample_data:
    if "::" in item:
        source, text = item.split("::", 1)
        documents.append({"source": source, "text": text})
    else:
        documents.append({"source": "未知", "text": item})

# 校验数据
if not documents:
    raise ValueError("无有效示例数据！")
print(f"加载完成，共{len(documents)}条FAISS相关知识点数据")

# ===================== 4. 生成文本嵌入向量 =====================
print("\n正在生成文本嵌入向量...")
# 提取文本内容
texts = [doc["text"] for doc in documents]
# 生成嵌入向量（归一化，适配内积索引）
embeddings = model.encode(
    texts,  # 简化写法，与原逻辑一致
    normalize_embeddings=True,
    convert_to_numpy=True  # 转换为numpy数组
).astype(np.float32)  # FAISS要求float32类型
print(f"嵌入向量生成完成，维度：{embeddings.shape}")

# ===================== 5. 创建并保存FAISS索引 =====================
# 初始化内积索引（归一化后等价于余弦相似度）
d = embeddings.shape[1]  # 向量维度
index = faiss.IndexFlatIP(d)
# 添加向量到索引
index.add(embeddings)
print(f"索引已添加向量数量：{index.ntotal}")

# 保存索引与元数据
faiss.write_index(index, str(index_path))
with open(metadata_path, "w", encoding="utf-8") as f:
    json.dump(documents, f, ensure_ascii=False, indent=2)
print(f"索引已保存至：{index_path}")
print(f"元数据已保存至：{metadata_path}")

# ===================== 6. 加载FAISS索引（模拟实际应用场景） =====================
print("\n正在加载索引与元数据...")
loaded_index = faiss.read_index(str(index_path))
with open(metadata_path, "r", encoding="utf-8") as f:
    loaded_metadata = json.load(f)
print("索引与元数据加载完成！")

# ===================== 7. 执行相似性检索（学生可替换不同查询文本测试） =====================
# 示例查询文本（学生可修改为其他问题，如"FAISS如何安装？"、"FAISS与Milvus的区别？"等）
query_text = "给出一首李白的七律？"
print(f"\n查询文本：{query_text}")

# 生成查询向量
query_embedding = model.encode(
    query_text,
    normalize_embeddings=True,
    convert_to_numpy=True
).astype(np.float32).reshape(1, -1)  # FAISS要求2D数组（batch_size, dim）

# 执行搜索
distances, indices = loaded_index.search(query_embedding, top_k)

# ===================== 8. 解析并展示结果 =====================
print("\n检索结果：")
# IndexFlatIP是内积计算，值越大相似度越高（归一化后为余弦相似度，范围[-1,1]）
for i in range(top_k):
    idx = indices[0][i]
    if idx == -1:  # 无结果时的处理
        print(f"排名{i+1}：无匹配结果")
        continue
    similarity = distances[0][i]
    print(f"排名{i+1}：相似度{similarity:.4f}")
    print(f"文本：{loaded_metadata[idx]['text']}")
    print(f"来源：{loaded_metadata[idx]['source']}\n")

d:\Users\PC\anaconda3\envs\faiss-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name ./model/iic/nlp_gte_sentence-embedding_chinese-base. Creating a new one with mean pooling.


正在加载模型...


Loading weights: 100%|██████████| 199/199 [00:01<00:00, 156.98it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: ./model/iic/nlp_gte_sentence-embedding_chinese-base
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


模型加载完成！
加载完成，共10条FAISS相关知识点数据

正在生成文本嵌入向量...
嵌入向量生成完成，维度：(10, 768)
索引已添加向量数量：10
索引已保存至：text_search_db\faiss_index.index
元数据已保存至：text_search_db\metadata.json

正在加载索引与元数据...
索引与元数据加载完成！

查询文本：给出一首李白的七律？

检索结果：
排名1：相似度0.7130
文本：登金陵凤凰台：凤凰台上凤凰游，凤去台空江自流。吴宫花草埋幽径，晋代衣冠成古丘。三山半落青天外，二水中分白鹭洲。总为浮云能蔽日，长安不见使人愁。
来源：未知

排名2：相似度0.7090
文本：早发白帝城：朝辞白帝彩云间，千里江陵一日还。两岸猿声啼不住，轻舟已过万重山。
来源：未知

排名3：相似度0.7026
文本：将进酒：君不见黄河之水天上来，奔流到海不复回。君不见高堂明镜悲白发，朝如青丝暮成雪。人生得意须尽欢，莫使金樽空对月。天生我材必有用，千金散尽还复来。
来源：未知



### FastApi搭建检索接口

阅读：[FastApi搭建检索接口](./ReadMe.md)